### RDR Calculation

In [ ]:
DicMap.plotMaxShear(plotGBs=True,vmin=0,vmax=10,plotColourBar=True,dilateBoundaries=False)

In [ ]:
### Install matplotlib_scalebar and pandas
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib as mpl
import pandas as pd
pixelinm=78.125e-9

def runRDR(x0,y0,x1,y1,length=5,cond=10,rcond=0,SelDicMap=DicMap):

    ulist=[]; vlist=[]; gradlist=[]; rsquaredlist=[]; index=[1]; id=[]; sliptraceno=[];

    ### Generate plots
    fig, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(12, 6))
    img = ax0.imshow(SelDicMap.crop(SelDicMap.max_shear),vmin=0,vmax=0.08, cmap='viridis')
    ### Set axis limits to only show region of interest
    ax0.set_xlim(np.amin([x0,x1])-50,np.amax([x0,x1])+50)
    ax0.set_ylim(np.amax([y0,y1])+50,np.amin([y0,y1])-50)
    ### Add scale bar, color bar and grain boundary maps
    ax0.add_artist(ScaleBar(pixelinm))
    fig.colorbar(img, ax=ax0,fraction=0.046, pad=0.04, label="Effective shear strain (%)")
    cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['white', 'white'], 256)
    cmap1._init()
    cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)
    boundariesImage = -SelDicMap.boundaries
    ax0.imshow(boundariesImage, cmap=cmap1, vmin=0, vmax=1)
    colors=['r','b','k','g','y','c','m']

    ### For each slip trace line
    for x0,y0,x1,y1 in zip(x0,y0,x1,y1):
        
        ### Plot numbered arrow
        ax0.arrow(x0, y0, x1-x0, y1-y0, head_width=5, head_length=5, fc='k', ec='k', lw=2)
        ax0.annotate(index[0], xy=(x0, y0), xycoords='data', xytext=(5, 5), textcoords='offset points')
        index[0]+=1

        ### Calculate properties for drawn line
        grad = (y1-y0)/(x1-x0)
        invgrad = -1/grad
        profile_length = np.sqrt((y1-y0)**2+(x1-x0)**2)
        num = np.round(1+profile_length*2)

        angle=np.arctan(np.abs(y1-y0)/np.abs(x1-x0))*180/3.141592654
        if grad>0:
            angle=180-angle
            
        print(angle)
        
        ### Calculate positions for each point along slip trace line (x,y)
        x, y = np.linspace(x0, x1, int(num)), np.linspace(y0, y1, int(num))
        x = np.round(x); y=np.round(y);
       
        ### Remove duplicate points
        df = pd.DataFrame({'x':x, 'y':y})
        df = df.drop_duplicates()
        x,y=df['x'].values.tolist(),df['y'].values.tolist()
        
        ### Plot
        ax0.plot(x,y, 'rx',lw=2)

        ### For each point along slip line
        for x,y in zip(x,y):

            ## Calculate positions for each point along lines perpendicular to slip line (xnew,ynew)
            x0new = x+np.sqrt(length/(invgrad**2+1))*np.sign(grad)
            y0new = y-np.sqrt(length/(1/invgrad**2+1))
            x1new = x-np.sqrt(length/(invgrad**2+1))*np.sign(grad)
            y1new = y+np.sqrt(length/(1/invgrad**2+1))
            profile_length=np.sqrt((y1new-y0new)**2+(x1new-x0new)**2)
            num = np.round(profile_length)
            xnew, ynew = np.linspace(x0new, x1new, int(num)), np.linspace(y0new, y1new, int(num))
            xnew, ynew=np.around(xnew).astype(int), np.around(ynew).astype(int)
            df = pd.DataFrame({'x':xnew, 'y':ynew})
            df = df.drop_duplicates()
            xnew,ynew=df['x'].values.tolist(),df['y'].values.tolist()

            ### Plot
            ax0.plot(xnew,ynew, 'rx',lw=0.5)
            ax0.plot(xnew,ynew, '-',lw=0.5)
            
            ### For all points, append u and v to list
            u =[]; v =[];
            for xnew,ynew in zip(xnew,ynew):
                u.append((SelDicMap.crop(SelDicMap.x_map))[ynew,xnew])
                v.append((SelDicMap.crop(SelDicMap.y_map))[ynew,xnew])
            
            ### Take away mean
            u=u-np.mean(u)
            v=v-np.mean(v)
            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x=v,y=u)

            ### Append to main arrays (ulist,vlist) (conditional upon u-centered, v-centered and individual rsquared-value)
            if all(cond >= i >= -cond for i in u) and all(cond >= i >= -cond for i in v) and r_value**2 > rcond:
                ulist.extend(u)
                vlist.extend(v)
                gradlist.append(slope)
                rsquaredlist.append(r_value**2)

    ### Linear regression of ucentered against vcentered
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x=vlist,y=ulist)

    ### Generate plots
    ax1.scatter(x=vlist,y=ulist,marker='x', lw=1)
    ax1.plot([np.min(vlist), np.max(vlist)],[slope*np.min(vlist)+intercept,slope*np.max(vlist)+intercept], '-')
    ax1.set_xlabel('v-centered')
    ax1.set_ylabel('u-centered')
    ax1.text(0.95, 0.01, 'Slope = {0:.3f} ± {1:.3f}\nR-squared = {2:.3f}\nn={3}'.format(slope,std_err,r_value**2,len(ulist)),
             verticalalignment='bottom', horizontalalignment='right', transform=ax1.transAxes, fontsize=10);

    ### Print regressions paramaters
    print("Slope {0:.3f} +- {1:.3f}".format(slope,std_err))
    print("R-squared: {0:.3f}".format(r_value**2))
    print("n: {0}".format(len(ulist)))

    ### Find grain
    Grain=SelDicMap.grainList[SelDicMap.grains[y0][x0]-1]
    print("Grain ID : {0}".format(SelDicMap.grains[y0][x0]-1))
    ebsdGrain = Grain.ebsdGrain

    ### Print euler angles for grain
    ebsdGrain.calcAverageSchmidFactors(loadVector=np.array([1, 0, 0]))
    ebsdGrain.calcSlipTraces()
    print("Euler angles: ")
    print("",ebsdGrain.refOri.eulerAngles()*180/np.pi, "\n")

    ### Print slip systems, directions, schmid, rdr
    schdFacs = ebsdGrain.averageSchmidFactors
    slipTraces = ebsdGrain.slipTraces
    RDRs=[]; sts=[]; sfs=[]
    for ssGroup, sfGroup, slipTrace in zip(EbsdMap.slipSystems, schdFacs, slipTraces):
        stAngle = np.arctan(slipTrace[1]/slipTrace[0]) *180/np.pi
        if stAngle < 0: stAngle += 180
        if stAngle > 180: stAngle -= 180
        sts.append(stAngle)
        print("Slip Plane Normal {0:s}\tSlip trace: ({1:.3f}, {2:.3f})\tAngle: {3:.1f}".
              format(ssGroup[0].slipPlaneLabel, slipTrace[0], slipTrace[1], stAngle))
        for ss, sf in zip(ssGroup, sfGroup):
            slipDirSample=ebsdGrain.refOri.conjugate.transformVector(ss.slipDir)
            print("\tSlip Direction {0:s}\tSchmid factor: {1:.3f}\tRDR: {2:.3f}".
                  format(ss.slipDirLabel, sf,-slipDirSample[0]/slipDirSample[1]))
            sfs.append(sf)
            RDRs.append(-slipDirSample[0]/slipDirSample[1])
    print(RDRs)
    
#     xvals=np.zeros(len(RDRs))
#     ax2.plot(xvals,RDRs,'o')
#     ax2.plot([0],slope,'ro')
#     ax2.set_ylim(-2,2)
#     ax2.set_xlim(-0.01,0.05)

#     ### Plot number line labels (only make sense for hexagonal)
#     labels=['','','','','','','','','','','','','','','','','','','','','','','','']
#     labels[0] = '<a> Ba {0:.1f} {1:.3f} Pr {2:.1f} {3:.3f} Py {4:.1f} {5:.3f} Py {6:.1f} {7:.3f}'.format(sts[0],sfs[0],sts[3],sfs[5],sts[4],sfs[6],sts[5],sfs[9])
#     labels[1] = str('<a> Ba {0:.1f} {1:.3f} Pr {2:.1f} {3:.3f} Py {4:.1f} {5:.3f} Py {6:.1f} {7:.3f}'.format(sts[0],sfs[1],sts[1],sfs[3],sts[6],sfs[12],sts[7],sfs[15]))
#     labels[2] = str('<a> Ba {0:.1f} {1:.3f} Pr {2:.1f} {3:.3f} Py {4:.1f} {5:.3f} Py {6:.1f} {7:.3f}'.format(sts[0],sfs[2],sts[2],sfs[4],sts[8],sfs[18],sts[9],sfs[21]))
#     labels[7] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[4],sfs[7],sts[6],sfs[14]))
#     labels[8] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[4],sfs[8],sts[8],sfs[20]))
#     labels[11] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[5],sfs[11],sts[7],sfs[17]))
#     labels[16] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[7],sfs[16],sts[8],sfs[19]))
#     labels[22] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[9],sfs[22],sts[6],sfs[13]))
#     labels[23] = str('<c+a> Py {0:.1f} {1:.3f} Py {2:.1f} {3:.3f}'.format(sts[9],sfs[23],sts[5],sfs[10]))

#     for i, txt in enumerate(labels):
#         ax2.annotate(txt, (xvals[i]+0.002,RDRs[i]-0.05))

#     ax2.annotate('{0:.3f}'.format(slope), (-0.009,slope-0.05))

#     labels=['','','','','','','','','','','','','','','','','','','','','','','','']
#     labels[0] = '{0:.3f}'.format(RDRs[0])
#     labels[1] = str('{0:.3f}'.format(RDRs[1]))
#     labels[2] = str('{0:.3f}'.format(RDRs[2]))
#     labels[7] = str('{0:.3f}'.format(RDRs[7]))
#     labels[8] = str('{0:.3f}'.format(RDRs[8]))
#     labels[11] = str('{0:.3f}'.format(RDRs[11]))
#     labels[16] = str('{0:.3f}'.format(RDRs[16]))
#     labels[22] = str('{0:.3f}'.format(RDRs[22]))
#     labels[23] = str('{0:.3f}'.format(RDRs[23]))

#     for i, txt in enumerate(labels):
#         ax2.annotate(txt, (-0.009,RDRs[i]-0.05))

    ### Get rid of whitespace
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.2)

In [ ]:
# Define the slip trace lines (x0,y0)->(x1,y1)
## Grain 20
x0=np.array([333])
y0=np.array([83])
x1=np.array([428])
y1=np.array([167])



runRDR(x0,y0,x1,y1,length=2.5, cond=10,rcond=0,SelDicMap=DicMap2)